In [2]:
import pandas as pd
from langchain_openai import ChatOpenAI
from ragas import evaluate
import httpx

from ragas.metrics import (
  context_precision,
  answer_relevancy,
  answer_correctness,
  context_recall,
)

In [ ]:
DATA_PATH = "./data/cleaned_resume.csv"
FAISS_PATH = "./vectorstore"
RAG_K_THRESHOLD = 2
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
LLM_MODEL = "gpt-35-turbo"
OPENAI_ENDPOINT = "https://aalto-openai-apigw.azure-api.net"
OPENAI_KEY = ""

In [ ]:
test_df = pd.read_csv("../data/main-data/synthetic-resumes-cp/")

In [ ]:
def update_base_url(request: httpx.Request):
  if request.url.path == "/chat/completions":
    request.url = request.url.copy_with(path="/v1/chat")

llm = ChatOpenAI(
    default_headers={"Ocp-Apim-Subscription-Key": OPENAI_KEY},
    base_url=OPENAI_ENDPOINT,
    api_key=False,
    http_client=httpx.Client(
      event_hooks={
        "request": [update_base_url],
    }),
  )

In [ ]:
result_df = evaluate(
  test_df,
  llm=llm,
  metrics=[
    answer_correctness,
    answer_relevancy,
    context_recall,
    context_precision,
]).to_pandas()

result_df.to_csv("./data/evaluation_result.csv", index=False)